In [5]:
from google.cloud import aiplatform, bigquery
from datetime import datetime
import pandas as pd
import numpy as np

In [25]:
project_id = "975291545274"
location = "us-central1"
endpoint_id = "676513011896156160"

aiplatform.init(project=project_id, location=location)
endpoint = aiplatform.Endpoint(endpoint_name=f"projects/{project_id}/locations/{location}/endpoints/{endpoint_id}")
bq_client = bigquery.Client(project=project_id)

df = pd.read_csv("./data/creditcard.csv")

In [27]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [45]:
df['log_amount'] = np.log1p(df['Amount'])
df_fraud = df[df['Class'] == 1].sample(n=10, random_state=42).drop(columns = ['Class']).sample(n=5, random_state=42)

In [46]:
df_fraud.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,log_amount
252124,155662.0,-1.928613,4.601506,-7.124053,5.716088,1.026579,-3.189073,-2.261897,1.185096,-4.441942,...,0.602291,-0.541287,-0.354639,-0.701492,-0.030973,0.034070,0.573393,0.294686,0.77,0.570980
12369,21662.0,-18.018561,10.558600,-24.667741,11.786180,-10.564657,-2.645681,-18.023468,2.693655,-6.219464,...,-2.319479,0.908839,1.352904,-1.059222,0.185751,0.687037,2.078081,-1.429517,1.00,0.693147
74794,55760.0,-6.003422,-3.930731,-0.007045,1.714669,3.414667,-2.329583,-1.901512,-2.746111,0.887673,...,1.101671,-0.992494,-0.698259,0.139898,-0.205151,-0.472412,1.775378,-0.104285,311.91,5.745916
17407,28692.0,-29.200329,16.155701,-30.013712,6.476731,-21.225810,-4.902997,-19.791248,19.168327,-3.617242,...,1.809371,-2.175815,-1.365104,0.174286,2.103868,-0.209944,1.278681,0.372393,99.99,4.615022
42674,41194.0,-7.896886,5.381020,-8.451162,7.963928,-7.862419,-2.376820,-11.949723,5.051356,-6.912076,...,2.557944,0.926278,0.032795,0.638073,0.361887,0.444577,1.101923,0.205958,1.52,0.924259


In [50]:
for index, row in df_fraud.iterrows():
    features = row.tolist()  
    
    # Predict
    response = endpoint.predict(instances=[features])
    prediction = int(response.predictions[0])

    # Log alert to BigQuery
    alert = {
        "transaction_id": index,
        "features": str(features),
        "prediction": prediction,
        "timestamp": datetime.utcnow().isoformat() + "Z"
    }

    table_id = f"{project_id}.fraud_detection_7.fraud_alerts"
    errors = bq_client.insert_rows_json(table_id, [alert])
    if errors:
        print(f"BigQuery error for transaction {index}:", errors)
    else:
        print(f"Logged prediction for {index}: {prediction}")

C:\Users\hutha\AppData\Local\Temp\ipykernel_13216\2941838920.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z"


Logged prediction for 252124: 1
Logged prediction for 12369: 1
Logged prediction for 74794: 1
Logged prediction for 17407: 1
Logged prediction for 42674: 1
